In [1]:
import os
import glob
import random
import numpy as np
import pandas as pd
from tqdm import tqdm
# import tempfile as tmpf
# import openpyxl as opyxl
# from openpyxl.utils.dataframe import dataframe_to_rows as df_to_rows

In [2]:
man_data_paths = '/home/jabblee/Desktop/CRC_collections/CRC_update/2023_Gatherings/2023_Gathering_m/Labeling/*.xlsx'
woman_data_paths = '/home/jabblee/Desktop/CRC_collections/CRC_update/2023_Gatherings/2023_Gathering_w/Labeling/*.xlsx'
total_data_path = '/home/jabblee/Desktop/CRC_collections/CRC_update/2023_Gatherings/'
save_train_data_path = total_data_path + 'TRAIN/'
save_test_data_path = total_data_path + 'TEST/'

os.makedirs(save_train_data_path, exist_ok = True)
os.makedirs(save_test_data_path, exist_ok = True)

man_xlsx_lst = sorted(glob.glob(man_data_paths))
woman_xlsx_lst = sorted(glob.glob(woman_data_paths))
total_xlsx_lst = man_xlsx_lst + woman_xlsx_lst
print('Total xlsx files :', len(total_xlsx_lst))

Total xlsx files : 12


In [3]:
# 총 자음 모음 수만큼의 sheet
sheet_name_lst = [('ㄱ', 1), ('ㄴ', 2), ('ㄷ', 3), ('ㄹ', 4), ('ㅁ', 5), ('ㅂ', 6), 
                  ('ㅅ', 7), ('ㅇ', 8), ('ㅈ', 9), ('ㅊ', 10), ('ㅋ', 11), ('ㅍ', 12), 
                  ('ㅎ', 13), ('ㅏ', 14), ('ㅑ', 15), ('ㅓ', 16), ('ㅕ', 17), 
                  ('ㅗ', 18), ('ㅛ', 19), ('ㅜ', 20), ('ㅡ', 21), ('ㅣ', 22)]

# # This is for making train and test datatseYaw	Pitch	Roll

# wb_test = opyxl.Workbook()
# train_xlsx_path = total_data_path + 'train.xlsx'
# test_xlsx_path = total_data_path + 'test.xlsx'
# wb_train.active.title = sheet_name_lst[0][0] + ' (' + str(sheet_name_lst[0][1]) + ')'
# wb_test.active.title = sheet_name_lst[0][0] + ' (' + str(sheet_name_lst[0][1]) + ')'

# for sheet_num, (hangul_character, label_cost) in enumerate(sheet_name_lst[1:]):
#     wb_train.create_sheet(hangul_character + ' (' + str(label_cost) + ')')
#     wb_test.create_sheet(hangul_character + ' (' + str(label_cost) + ')')
    
#     wb_train.save(train_xlsx_path)
#     wb_test.save(test_xlsx_path)

class_diagrams = [[] for _ in range(len(sheet_name_lst) + 1)]
class_diagrams_dict = dict.fromkeys([hangul_chr for hangul_chr, label_cst in sheet_name_lst])

# train_tmp = tmpf.TemporaryFile(mode = 'a+b')
total_people_train_fingers = []
total_people_train_accelerates = []
for person_idx, person_data_path in enumerate(total_xlsx_lst):
    person_name = person_data_path.split('/')[-1].replace('.xlsx', '')
    # print('person_name :', person_name)

    person_train_fingers = []
    person_train_accelerates = []
    for sheet_num, (hangul_character, label_cost) in tqdm(enumerate(sheet_name_lst)):
        person_sheet_total = pd.read_excel(person_data_path, sheet_name = sheet_num,
                                           header = 2)
        person_sheet_total = person_sheet_total.iloc[:, 2:]
        
        person_sheet_label_cost = person_sheet_total.columns[-1]
        if person_sheet_label_cost == hangul_character:
            person_sheet_total.rename(columns = {hangul_character:'Label'}, inplace = True)
            
        person_sheet_valid = person_sheet_total.loc[(person_sheet_total['Label'] == label_cost) |
                                                    (person_sheet_total['Label'] == 0)]
        person_sheet_valid = person_sheet_valid.loc[:, ['Thumb', 'Index', 'Middle', 'Ring', 'Little', 'Yaw', 'Pitch', 'Roll', 'Label']]
        
        # Erase NaN values
        person_sheet_valid = person_sheet_valid.dropna(axis = 0)
        print('person_sheet_valid Shape :', person_sheet_valid.shape)
        
        # Divide 0 and label_cost in valid DataFrame
        person_sheet_valid_zeros = person_sheet_valid.loc[person_sheet_valid['Label'] == 0]
        person_sheet_valid_label_costs = person_sheet_valid.loc[person_sheet_valid['Label'] != 0]
        print('person_sheet_valid_zeros Shape :', person_sheet_valid_zeros.shape)
        print('person_sheet_valid_label_costs Shape :', person_sheet_valid_label_costs.shape)        


        person_sheet_valid_zeros_train = person_sheet_valid_zeros.sample(frac = 0.8, random_state = 7)
        person_sheet_valid_zeros_test = pd.merge(person_sheet_valid_zeros, person_sheet_valid_zeros_train,
                                                 how = 'outer',
                                                 indicator = True).query(
                                                     '_merge == "left_only"'
                                                     ).drop(columns = ['_merge'])
        
        person_sheet_valid_label_costs_train = person_sheet_valid_label_costs.sample(frac = 0.8, random_state = 7)
        person_sheet_valid_label_costs_test = pd.merge(person_sheet_valid_label_costs, person_sheet_valid_label_costs_train,
                                                       how = 'outer',
                                                       indicator = True).query(
                                                           '_merge == "left_only"'
                                                       ).drop(columns = ['_merge'])
        
        
        
        person_sheet_train = pd.concat([person_sheet_valid_zeros_train, person_sheet_valid_label_costs_train])
        person_sheet_test = pd.concat([person_sheet_valid_zeros_test, person_sheet_valid_label_costs_test])
        
        person_sheet_train_np = person_sheet_train.to_numpy()
        person_sheet_test_np = person_sheet_test.to_numpy()
        print(person_sheet_train_np.shape, person_sheet_test_np.shape)
        
        # Divide ndarray with 40 rows
        train_total_rows, train_total_columns = person_sheet_train_np.shape
        test_total_rows, test_total_columns = person_sheet_test_np.shape
        
        train_quota, _ = divmod(train_total_rows, 40)
        test_quota, _ = divmod(test_total_rows, 40)

        # for train_cnt in range(train_quota + 1):
        #     chunk_person_sheet_train_np = person_sheet_train_np[train_cnt * 40:train_cnt * 40 + 40, :]
        #     np.save(save_train_data_path + person_name + '_' + str(label_cost) + '_' + str(train_cnt), chunk_person_sheet_train_np)

        # for test_cnt in range(test_quota + 1):
        #     chunk_person_sheet_test_np = person_sheet_test_np[test_cnt * 40:test_cnt * 40 + 40, :]
        #     np.save(save_test_data_path + person_name + '_' + str(label_cost) + '_' + str(test_cnt), chunk_person_sheet_test_np)

print('Done for making Pandas DataFrame to Numpy npy file.')

0it [00:00, ?it/s]

3it [00:00,  7.71it/s]

person_sheet_valid Shape : (263, 9)
person_sheet_valid_zeros Shape : (119, 9)
person_sheet_valid_label_costs Shape : (144, 9)
(210, 9) (53, 9)
person_sheet_valid Shape : (275, 9)
person_sheet_valid_zeros Shape : (118, 9)
person_sheet_valid_label_costs Shape : (157, 9)
(220, 9) (55, 9)
person_sheet_valid Shape : (269, 9)
person_sheet_valid_zeros Shape : (115, 9)
person_sheet_valid_label_costs Shape : (154, 9)
(215, 9) (54, 9)


5it [00:00,  8.07it/s]

person_sheet_valid Shape : (258, 9)
person_sheet_valid_zeros Shape : (109, 9)
person_sheet_valid_label_costs Shape : (149, 9)
(206, 9) (52, 9)
person_sheet_valid Shape : (301, 9)
person_sheet_valid_zeros Shape : (133, 9)
person_sheet_valid_label_costs Shape : (168, 9)
(240, 9) (61, 9)


...

person_sheet_valid Shape : (278, 9)
person_sheet_valid_zeros Shape : (110, 9)
person_sheet_valid_label_costs Shape : (168, 9)
(222, 9) (56, 9)
person_sheet_valid Shape : (329, 9)
person_sheet_valid_zeros Shape : (119, 9)
person_sheet_valid_label_costs Shape : (210, 9)
(263, 9) (66, 9)
person_sheet_valid Shape : (269, 9)
person_sheet_valid_zeros Shape : (87, 9)
person_sheet_valid_label_costs Shape : (182, 9)
(216, 9) (53, 9)


21it [00:02,  9.80it/s]

person_sheet_valid Shape : (280, 9)
person_sheet_valid_zeros Shape : (107, 9)
person_sheet_valid_label_costs Shape : (173, 9)
(224, 9) (56, 9)
person_sheet_valid Shape : (275, 9)
person_sheet_valid_zeros Shape : (115, 9)
person_sheet_valid_label_costs Shape : (160, 9)
(220, 9) (55, 9)


22it [00:02,  9.17it/s]

person_sheet_valid Shape : (281, 9)
person_sheet_valid_zeros Shape : (97, 9)
person_sheet_valid_label_costs Shape : (184, 9)
(225, 9) (56, 9)
Done for making Pandas DataFrame to Numpy npy file.
